# [모듈 3.1] Code Pipeline 생성 하기

## 참고 자료
- https://docs.aws.amazon.com/codepipeline/latest/userguide/pipelines-create.html


# 1. 환경 설정


In [69]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append('./src')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1.0 기존의 변수 불러오기

In [70]:
%store -r code_pipeline_serving_config_json_path
%store -r sm_pipeline_serving_config_json_path

from common_utils import load_json

code_pipeline_serving_dict = load_json(code_pipeline_serving_config_json_path)
sm_pipeline_serving_dict = load_json(sm_pipeline_serving_config_json_path)

import json
print("Code Pipeline Series Params: ")
print (json.dumps(code_pipeline_serving_dict, indent=2))
print("SageMaker Pipeline Series Params: ")
print (json.dumps(sm_pipeline_serving_dict, indent=2))



Code Pipeline Series Params: 
{
  "code_pipeline_role_arn": "arn:aws:iam::057716757052:role/code-pipeline-gsmoon",
  "code_build_service_arn": "arn:aws:iam::057716757052:role/codebuild-gsmoon",
  "project_prefix": "CodePipeline-Serving-NCF",
  "region": "us-east-1",
  "account_id": "057716757052",
  "serving_code_repo_name": "ncf-serving",
  "code_build_project_name": "ncf-serving-sm-pipeline",
  "bucket": "sagemaker-us-east-1-057716757052",
  "code_pipeline_name": "ncf-serving-code-pipeline",
  "model_package_group_name": "NCF-Model-CodePipeline",
  "endpoint_name": "ncf-codepipeline-endpoint",
  "branch_name": "master"
}
SageMaker Pipeline Series Params: 
{
  "sm_pipeline_name": "sm-serving-pipeline",
  "endpoint_instance_type": "ml.g4dn.xlarge",
  "ModelApprovalStatus": "Approved",
  "bucket": "sagemaker-us-east-1-057716757052"
}


# 2. 변수 정의

In [72]:
region = code_pipeline_serving_dict["region"]
repository_name = code_pipeline_serving_dict["serving_code_repo_name"]
code_pipeline_name = code_pipeline_serving_dict["code_pipeline_name"]
code_build_project_name = code_pipeline_serving_dict["code_build_project_name"]
branch_name = code_pipeline_serving_dict["branch_name"]
pipeline_role_name = code_pipeline_serving_dict["code_pipeline_role_arn"]
bucket = code_pipeline_serving_dict["bucket"]


# 4. 파이프라인 JSON 정의

In [73]:
pipeline_dic = {
    "pipeline": {
        "name": code_pipeline_name,
        "roleArn": pipeline_role_name,
        "artifactStore": {
            "type": "S3",
            "location": bucket
        },
        "stages": [
            {
                "name": "Source",
                "actions": [
                    {
                        "name": "Source",
                        "actionTypeId": {
                            "category": "Source",
                            "owner": "AWS",
                            "provider": "CodeCommit",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "BranchName": branch_name,
                            "OutputArtifactFormat": "CODE_ZIP",
                            "PollForSourceChanges": "false",
                            "RepositoryName": repository_name
                        },
                        "outputArtifacts": [
                            {
                                "name": "SourceArtifact"
                            }
                        ],
                        "inputArtifacts": [],
                        "region": region,
                        "namespace": "SourceVariables"
                    }
                ]
            },
            {
                "name": "Build",
                "actions": [
                    {
                        "name": "BuildAndExecuteSageMakerPipeline",
                        "actionTypeId": {
                            "category": "Build",
                            "owner": "AWS",
                            "provider": "CodeBuild",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "ProjectName": code_build_project_name
                        },
                        "outputArtifacts": [
                            {
                                "name": "BuildArtifact"
                            }
                        ],
                        "inputArtifacts": [
                            {
                                "name": "SourceArtifact"
                            }
                        ],
                        "region": region,
                        "namespace": "BuildVariables"
                    }
                ]
            }
        ],
        "version": 1
    },
}

## 4.1. Dic 포맷을 JSON 으로 변경

In [74]:
import json

pipeline_json = json.dumps(pipeline_dic)
json.loads(pipeline_json)

{'pipeline': {'name': 'ncf-serving-code-pipeline',
  'roleArn': 'arn:aws:iam::057716757052:role/code-pipeline-gsmoon',
  'artifactStore': {'type': 'S3',
   'location': 'sagemaker-us-east-1-057716757052'},
  'stages': [{'name': 'Source',
    'actions': [{'name': 'Source',
      'actionTypeId': {'category': 'Source',
       'owner': 'AWS',
       'provider': 'CodeCommit',
       'version': '1'},
      'runOrder': 1,
      'configuration': {'BranchName': 'master',
       'OutputArtifactFormat': 'CODE_ZIP',
       'PollForSourceChanges': 'false',
       'RepositoryName': 'ncf-serving'},
      'outputArtifacts': [{'name': 'SourceArtifact'}],
      'inputArtifacts': [],
      'region': 'us-east-1',
      'namespace': 'SourceVariables'}]},
   {'name': 'Build',
    'actions': [{'name': 'BuildAndExecuteSageMakerPipeline',
      'actionTypeId': {'category': 'Build',
       'owner': 'AWS',
       'provider': 'CodeBuild',
       'version': '1'},
      'runOrder': 1,
      'configuration': {'Projec

## 4.2. JSON 파일 저장

In [75]:
json_file_path = 'src/hello_pipeline.json'
with open(json_file_path, "w") as outfile:
    outfile.write(pipeline_json)

In [76]:
with open(json_file_path) as json_file:
    json_data = json.load(json_file)
    
json_data    

{'pipeline': {'name': 'ncf-serving-code-pipeline',
  'roleArn': 'arn:aws:iam::057716757052:role/code-pipeline-gsmoon',
  'artifactStore': {'type': 'S3',
   'location': 'sagemaker-us-east-1-057716757052'},
  'stages': [{'name': 'Source',
    'actions': [{'name': 'Source',
      'actionTypeId': {'category': 'Source',
       'owner': 'AWS',
       'provider': 'CodeCommit',
       'version': '1'},
      'runOrder': 1,
      'configuration': {'BranchName': 'master',
       'OutputArtifactFormat': 'CODE_ZIP',
       'PollForSourceChanges': 'false',
       'RepositoryName': 'ncf-serving'},
      'outputArtifacts': [{'name': 'SourceArtifact'}],
      'inputArtifacts': [],
      'region': 'us-east-1',
      'namespace': 'SourceVariables'}]},
   {'name': 'Build',
    'actions': [{'name': 'BuildAndExecuteSageMakerPipeline',
      'actionTypeId': {'category': 'Build',
       'owner': 'AWS',
       'provider': 'CodeBuild',
       'version': '1'},
      'runOrder': 1,
      'configuration': {'Projec

# 5. 파이프라인 실행

In [77]:
%%sh -s {json_file_path}
json_file_path=$1
echo $json_file_path
echo $PWD
aws codepipeline create-pipeline --cli-input-json file://$json_file_path
# aws codepipeline create-pipeline --cli-input-json file://src/hello_pipeline.json

src/hello_pipeline.json
/home/ec2-user/SageMaker/Neural-Collaborative-Filtering-On-SageMaker/3_MLOps/5_sm-serving-codepipeline
{
    "pipeline": {
        "name": "ncf-serving-code-pipeline",
        "roleArn": "arn:aws:iam::057716757052:role/code-pipeline-gsmoon",
        "artifactStore": {
            "type": "S3",
            "location": "sagemaker-us-east-1-057716757052"
        },
        "stages": [
            {
                "name": "Source",
                "actions": [
                    {
                        "name": "Source",
                        "actionTypeId": {
                            "category": "Source",
                            "owner": "AWS",
                            "provider": "CodeCommit",
                            "version": "1"
                        },
                        "runOrder": 1,
                        "configuration": {
                            "BranchName": "master",
                            "OutputArtifactFormat": "COD

# 5. 코드 파이프라인 실행 상태 확인

In [78]:
%%sh -s {code_pipeline_name}
code_pipeline_name=$1
echo $code_pipeline_name
aws codepipeline get-pipeline-state --name $code_pipeline_name

ncf-serving-code-pipeline
{
    "pipelineName": "ncf-serving-code-pipeline",
    "pipelineVersion": 1,
    "stageStates": [
        {
            "stageName": "Source",
            "inboundTransitionState": {
                "enabled": true
            },
            "actionStates": [
                {
                    "actionName": "Source",
                    "latestExecution": {
                        "actionExecutionId": "589c4a0f-8545-46f1-bc6e-8ec39554ea3d",
                        "status": "InProgress",
                        "lastStatusChange": 1668946178.541
                    },
                    "entityUrl": "https://console.aws.amazon.com/codecommit/home?region=us-east-1#/repository/ncf-serving/browse/master/--/"
                }
            ],
            "latestExecution": {
                "pipelineExecutionId": "bc533b14-e222-4699-b28d-014795d7a5aa",
                "status": "InProgress"
            }
        },
        {
            "stageName": "Build",
  